# Requests and Responses

Agents interact with the PrimAITE simulation via the Request system.

© Crown-owned copyright 2024, Defence Science and Technology Laboratory UK

## Sending a request

Let's set up a minimal network simulation and send some requests to see how it works.

In [ ]:
from primaite.simulator.network.hardware.node_operating_state import NodeOperatingState
from primaite.simulator.network.hardware.nodes.host.host_node import HostNode
from primaite.simulator.sim_container import Simulation


In [ ]:
sim = Simulation()
sim.network.add_node(
    HostNode(
        hostname="client",
        ip_address='10.0.0.1',
        subnet_mask='255.255.255.0',
        operating_state=NodeOperatingState.ON)
)
client = sim.network.get_node_by_hostname('client')


A request is structured in a similar way to a command line interface - a list of strings with positional args. It's also possible to supply an optional `context` dictionary. We will craft a request that stops the pre-installed DNSClient service on the client node.

First let's verify that the DNS Client is running on the client.


In [ ]:
client.software_manager.show()

Send a request to the simulator to stop the DNSClient.

In [ ]:
response = sim.apply_request(
    request=["network", "node", "client", "service", "DNSClient", "stop"],
    context={}
    )
print(response)


The request returns a `RequestResponse` object which tells us that the request was successfully executed. Let's verify that the DNS client is in a stopped state now.

In [ ]:
print(f"DNS Client state: {client.software_manager.software.get('DNSClient').operating_state.name}")

## Unreachable requests

If we attempt to send a request to something that doesn't exist, we will get an unreachable request status.

In [ ]:
response = sim.apply_request(
    request=["network", "node", "client", "service", "NonExistentApplication", "stop"],
    context={}
    )
print(response)

## Failed requests

Sometimes requests cannot be executed by the simulation. For example if we turn off the client node, we cannot execute the software that is running on it.

In [ ]:
response = sim.apply_request(
    request = ["network", "node", "client", "shutdown"],
    context = {}
)
print(response)

We need to apply timestep a few times for the client to go from `SHUTTING_DOWN` to `OFF` state.

In [ ]:
print(f"client is in state: {client.operating_state.name}")
sim.apply_timestep(1)
sim.apply_timestep(2)
sim.apply_timestep(3)
sim.apply_timestep(4)
print(f"client is in state: {client.operating_state.name}")

Now, if we try to start the DNSClient back up, we get a failure because we cannot start software on a node that is turned off.

In [ ]:
response = sim.apply_request(
    request=["network", "node", "client", "service", "DNSClient", "start"],
    context={}
    )
print(response)